In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import numpy as np
import statsmodels.api as sm
from sklearn.cluster import KMeans
import scikit_posthocs as sp
from scipy.stats import kruskal
from sklearn.utils import resample
from sklearn.feature_selection import RFE
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

pd.set_option('display.max_columns', None)  # Отображение всех столбцов
# pd.set_option('display.max_rows', None)  # Ото----бражение всех строк   
regions = pd.read_csv('regions.csv')
starts = pd.read_csv('starts.csv')
volunteers = pd.read_csv('volunteers.csv')



volunteers = volunteers[volunteers.athlete_age < 100]
starts = starts[starts.athlete_age < 80]
starts = starts[starts.finish_time < 5000]

starts['rain.1h'] = starts['rain.1h'].fillna(0)
starts['snow.1h'] = starts['snow.1h'].fillna(0)

cnt_ludei = starts.groupby(['event_name', 'event_date', 'region']).agg(
    cnt_runners = ('athlete_id', 'count')
)


young_volunteers = volunteers[volunteers['athlete_age'] <= 14]

starts = starts.merge(young_volunteers[['event_name', 'event_date', 'athlete_id']], 
                      on=['event_name', 'event_date'], 
                        how='left',
                      suffixes=('', '_vol'))

starts['has_young_volunteers'] = starts['athlete_id_vol'].notna()
starts['has_young_volunteers'] =starts['has_young_volunteers'].astype(int)
starts = starts.drop(columns=['athlete_id_vol'], axis=1)





ages_14 = starts[starts['athlete_age'] == 14]['athlete_id'].unique()
ages_15 = starts[starts['athlete_age'] == 15]['athlete_id'].unique()
common_athletes = set(ages_14) & set(ages_15)
starts = starts[~starts['athlete_id'].isin(common_athletes)]



starts['is_male'] = starts['athlete_gender'].apply(lambda x: 1 if x == 'мужской' else 0)
starts = starts[starts['athlete_age'] > 14]

volunteer_counts = volunteers.groupby(['event_date','event_name', 'volunteer_role_name']).size().unstack(fill_value=0)  
starts = starts.merge(volunteer_counts, on=['event_date','event_name'], how='left')  

starts = starts.dropna(subset=[col for col in starts.columns if col not in ['snow.1h', 'rain.1h']])   # удаление где 0 волонтеров в сумме было


unique_dates = (
    pd.concat([
        starts[['athlete_id', 'event_date']],
        volunteers[['athlete_id', 'event_date']]
    ])
    .drop_duplicates()
)
date_counts = unique_dates.groupby('athlete_id').size().reset_index(name='unique_event_count')
date_counts['is_repeated'] = (date_counts['unique_event_count'] > 1).astype(int)
starts = starts.merge(date_counts[['athlete_id', 'is_repeated']], on='athlete_id', how='left')


# starts = starts[~starts['region'].isin(['Республика Адыгея', 'Республика Карелия', 'Липецкая область', 'Сахалинская область', 'Архангельская область'])]  
starts = starts[starts.event_date <= '2023-10-15']


newbie = starts[starts['is_newbie'] == 1]
first_runs = newbie.sort_values(by='event_date').drop_duplicates('athlete_id')
didnt_come = first_runs[first_runs.is_repeated == 0]

come = first_runs[first_runs.is_repeated == 1]
come_with_didnt_come = pd.concat([come, didnt_come])
come_with_didnt_come['cnt_volunteers'] = come_with_didnt_come.iloc[:, 25:52].sum(axis=1)
come_with_didnt_come = come_with_didnt_come.merge(cnt_ludei,on=['event_name', 'event_date', 'region'], how="left")
print(len(come_with_didnt_come))

16708


ОБЩИЙ КАТБУСТ!

In [2]:
from catboost import CatBoostClassifier, Pool  
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split



X = come_with_didnt_come.drop(columns=['is_repeated', 'athlete_id', 'record_id', 'event_name', 'event_date', 'region', 'city', 'is_newbie', 
   # 'athlete_gender', 'weather.main','weather.description','track_description_surface',
 #   'dew_point',
    'is_male',
    'cnt_volunteers',
    #'humidity','finish_time', 'athlete_age', 'athlete_gender', 'wind_speed', 'feels_like', 
    #'pressure', 'temp', 
    # 'snow.1h', 'rain.1h', 'weather.main', 'weather.description', 'track_description_surface', 'wind_deg', 'clouds', 
    #'cnt_runners'
], axis=1)  
y = come_with_didnt_come['is_repeated']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42) 
categorical_cols = ['athlete_gender', 'weather.main', 'weather.description', 'track_description_surface']

model = CatBoostClassifier(iterations=1000, depth=4, learning_rate=0.1,verbose=0  ,cat_features=categorical_cols)  
print('катбуст начальный со всеми переменными')
model.fit(X, y)  


preds = model.predict(X_test)  
cm = confusion_matrix(y_test, preds)
print(cm)
print(f'Accuracy: {accuracy_score(y_test, preds):.2f}') 




feature_importances = model.get_feature_importance()  
importance_df = pd.DataFrame({  
    'Feature': X.columns,  # Используйте реальные имена признаков  
    'Importance': feature_importances  
})  

# Сортировка значений важности в порядке убывания  
importance_df = importance_df.sort_values(by='Importance', ascending=False)  

display(importance_df)


катбуст начальный со всеми переменными
[[417 331]
 [151 772]]
Accuracy: 0.71


,Feature,Importance
2,athlete_age,11.513688
0,finish_time,10.882341
44,cnt_runners,8.617707
9,wind_speed,6.232879
38,Сканирование штрих-кодов,5.454182
3,temp,5.410655
7,dew_point,5.198736
10,wind_deg,5.087613
5,pressure,4.866334
6,humidity,4.584611


# КАТБУСТ ДЛЯ 1 ГРУППЫ (2-46 УЧАСТНИКОВ)

In [ ]:
from catboost import CatBoostClassifier, Pool  
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split


X = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 2) & (come_with_didnt_come.cnt_runners <= 46)]
y = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 2) & (come_with_didnt_come.cnt_runners <= 46)]

X = X.drop(columns=['is_repeated', 'athlete_id', 'record_id', 'event_name', 'event_date', 'region', 'city', 'is_newbie', 
    'athlete_gender', 'weather.main','weather.description','track_description_surface',
   'dew_point','wind_deg', 'clouds', 'temp',
    'is_male',
    'cnt_volunteers', 'rain.1h', 'snow.1h',

    'finish_time', 'pressure', 'humidity','wind_speed','feels_like', 'cnt_runners', 'athlete_age',

    #'humidity','finish_time', 'athlete_age', 'athlete_gender', 'wind_speed', 'feels_like', 
    #'pressure', 'temp', 
    # 'snow.1h', 'rain.1h', 'weather.main', 'weather.description', 'track_description_surface', 'wind_deg', 'clouds', 
    #'cnt_runners'
], axis=1)  
y = y['is_repeated']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42) 
#categorical_cols = ['athlete_gender', 'weather.main', 'weather.description', 'track_description_surface']

model = CatBoostClassifier(iterations=1000, depth=4, learning_rate=0.1,verbose=0 )# ,cat_features=categorical_cols)  
print('катбуст для группы 2-46 УЧАСТНИКОВ для волонтеров')
model.fit(X, y)  


preds = model.predict(X_test)  
cm = confusion_matrix(y_test, preds)
print(cm)
print(f'Accuracy: {accuracy_score(y_test, preds):.2f}') 




feature_importances = model.get_feature_importance()  
importance_df = pd.DataFrame({  
    'Feature': X.columns,  # Используйте реальные имена признаков  
    'Importance': feature_importances  
})  

# Сортировка значений важности в порядке убывания  
importance_df = importance_df.sort_values(by='Importance', ascending=False)  

display(importance_df)


катбуст для группы 2-46 УЧАСТНИКОВ для волонтеров
[[ 79 251]
 [ 32 450]]
Accuracy: 0.65


,Feature,Importance
8,Маршал,10.233471
26,Фотограф,9.594812
27,Хранение и доставка оборудования,7.174089
13,Подготовка мероприятия,6.742984
17,Проверка трассы,5.728788
20,Связи с общественностью,5.062199
0,has_young_volunteers,4.383847
19,Разное,4.356270
21,Секундомер,4.097157
9,Обработка результатов,4.034061


Берем из этого катбуста топ-10 волонтеров и контрольные переменные. модель для 1 группы 2-46 участников

In [ ]:
X = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 2) & (come_with_didnt_come.cnt_runners <= 46)]
y = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 2) & (come_with_didnt_come.cnt_runners <= 46)]
print(len(come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 2) & (come_with_didnt_come.cnt_runners <= 46)].drop_duplicates(['event_name', 'event_date'])))


need_columns = ['finish_time', 'athlete_age', 'feels_like', 'pressure', 'humidity', 'wind_speed', 
                'Маршал', 'Фотограф','Хранение и доставка оборудования', 'Подготовка мероприятия','Проверка трассы', 'Связи с общественностью','has_young_volunteers',
                  'Разное','Секундомер', 'Обработка результатов','Проведение разминки','Инструктаж новых участников', 'Сортировка карточек',
                  'Замыкающий', 'Организация финиша',
               # '', '','', '','', '','', '',
                  'cnt_runners'
                  ]
                






for col1 in need_columns:
    for col2 in need_columns:
        if col1 != col2:
            try:
                coef = X[col1].corr(X[col2],method='spearman')
                if abs(coef) > 0.8:
                    print('Возможна мультиколлинеарность: ', coef, col1, col2)
            except:
                print('ошибка в цикле на проверке корреляций между ', col1, " ", col2)

for_model = X[need_columns]


for_model = sm.add_constant(for_model)
y = y['is_repeated']
model = sm.Logit(y, for_model).fit()
print('\n')
print('Модель для предсказывания is_repeated новичка, в пробежках где было 2-46 людей')
print('\n')
print(model.summary())
marginal_effects = model.get_margeff()
print('\n')
print('Предельные эффекты:')
print(marginal_effects.summary())
print('\n')

y_pred = model.predict(for_model)
y_pred_class = (y_pred > 0.5).astype(int)  
cm = confusion_matrix(y, y_pred_class)
print(cm)
accuracy = (y_pred_class == y).mean()  
print(f'Accuracy: {accuracy:.2f}')  

2591
Optimization terminated successfully.
         Current function value: 0.659797
         Iterations 5


Модель для предсказывания is_repeated новичка, в пробежках где было 2-46 людей


                           Logit Regression Results                           
Dep. Variable:            is_repeated   No. Observations:                 8113
Model:                          Logit   Df Residuals:                     8091
Method:                           MLE   Df Model:                           21
Date:                Tue, 10 Dec 2024   Pseudo R-squ.:                 0.02499
Time:                        14:05:52   Log-Likelihood:                -5352.9
converged:                       True   LL-Null:                       -5490.1
Covariance Type:            nonrobust   LLR p-value:                 4.956e-46
                                       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------

Только один волонтер стат значим (хранение и доставка оборудования)

# КАТБУСТ ДЛЯ 2 ГРУППЫ (47-138 УЧАСТНИКОВ)

In [12]:
from catboost import CatBoostClassifier, Pool  
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split


X = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 47) & (come_with_didnt_come.cnt_runners <= 138)]
y = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 47) & (come_with_didnt_come.cnt_runners <= 138)]

X = X.drop(columns=['is_repeated', 'athlete_id', 'record_id', 'event_name', 'event_date', 'region', 'city', 'is_newbie', 
    'athlete_gender', 'weather.main','weather.description','track_description_surface',
   'dew_point','wind_deg', 'clouds', 'temp',
    'is_male',
    'cnt_volunteers', 'rain.1h', 'snow.1h',

    'finish_time', 'pressure', 'humidity','wind_speed','feels_like', 'cnt_runners', 'athlete_age',

    #'humidity','finish_time', 'athlete_age', 'athlete_gender', 'wind_speed', 'feels_like', 
    #'pressure', 'temp', 
    # 'snow.1h', 'rain.1h', 'weather.main', 'weather.description', 'track_description_surface', 'wind_deg', 'clouds', 
    #'cnt_runners'
], axis=1)  
y = y['is_repeated']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42) 
#categorical_cols = ['athlete_gender', 'weather.main', 'weather.description', 'track_description_surface']

model = CatBoostClassifier(iterations=1000, depth=4, learning_rate=0.1,verbose=0 )# ,cat_features=categorical_cols)  
print('катбуст для группы 47-138 УЧАСТНИКОВ для волонтеров')
model.fit(X, y)  


preds = model.predict(X_test)  
cm = confusion_matrix(y_test, preds)
print(cm)
print(f'Accuracy: {accuracy_score(y_test, preds):.2f}') 




feature_importances = model.get_feature_importance()  
importance_df = pd.DataFrame({  
    'Feature': X.columns,  # Используйте реальные имена признаков  
    'Importance': feature_importances  
})  

# Сортировка значений важности в порядке убывания  
importance_df = importance_df.sort_values(by='Importance', ascending=False)  

display(importance_df)


катбуст для группы 47-138 УЧАСТНИКОВ для волонтеров
[[120 132]
 [ 74 258]]
Accuracy: 0.65


,Feature,Importance
8,Маршал,9.216642
13,Подготовка мероприятия,7.824500
20,Связи с общественностью,6.660053
26,Фотограф,6.146613
22,Сканирование штрих-кодов,5.903290
27,Хранение и доставка оборудования,5.485024
9,Обработка результатов,4.716427
16,Проверка карточек позиций,4.392179
3,Замыкающий,4.119215
23,Сортировка карточек,4.089783


Берем из этого катбуста топ-10 волонтеров и контрольные переменные. модель для 2 группы 47-138 участников

In [21]:
X = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 47) & (come_with_didnt_come.cnt_runners <= 138)]
y = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 47) & (come_with_didnt_come.cnt_runners <= 138)]



need_columns = ['finish_time', 'athlete_age', 'feels_like', 'pressure', 'humidity', 'wind_speed', 
                'Маршал', 'Подготовка мероприятия','Связи с общественностью', 'Фотограф','Сканирование штрих-кодов', 'Хранение и доставка оборудования','Обработка результатов',
                  'Проверка карточек позиций','Замыкающий', 'Сортировка карточек',
               # 'Организация финиша', 'Разное','Проверка трассы', 'Координация волонтёров','Инструктаж новых участников',
                 # '','', '',
                  'cnt_runners'
                  ]
                






for col1 in need_columns:
    for col2 in need_columns:
        if col1 != col2:
            try:
                coef = X[col1].corr(X[col2],method='spearman')
                if abs(coef) > 0.8:
                    print('Возможна мультиколлинеарность: ', coef, col1, col2)
            except:
                print('ошибка в цикле на проверке корреляций между ', col1, " ", col2)

for_model = X[need_columns]


for_model = sm.add_constant(for_model)
y = y['is_repeated']
model = sm.Logit(y, for_model).fit()
print('\n')
print('Модель для предсказывания is_repeated новичка, в пробежках где было 47-138 людей')
print('\n')
print(model.summary())
marginal_effects = model.get_margeff()
print('\n')
print('Предельные эффекты:')
print(marginal_effects.summary())
print('\n')

y_pred = model.predict(for_model)
y_pred_class = (y_pred > 0.5).astype(int)  
cm = confusion_matrix(y, y_pred_class)
print(cm)
accuracy = (y_pred_class == y).mean()  
print(f'Accuracy: {accuracy:.2f}')  

Optimization terminated successfully.
         Current function value: 0.678012
         Iterations 4


Модель для предсказывания is_repeated новичка, в пробежках где было 47-138 людей


                           Logit Regression Results                           
Dep. Variable:            is_repeated   No. Observations:                 5836
Model:                          Logit   Df Residuals:                     5818
Method:                           MLE   Df Model:                           17
Date:                Tue, 10 Dec 2024   Pseudo R-squ.:                 0.01461
Time:                        14:10:58   Log-Likelihood:                -3956.9
converged:                       True   LL-Null:                       -4015.5
Covariance Type:            nonrobust   LLR p-value:                 4.988e-17
                                       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------

# КАТБУСТ ДЛЯ 3 ГРУППЫ (139-404 УЧАСТНИКОВ)

In [14]:
from catboost import CatBoostClassifier, Pool  
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split


X = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 139) & (come_with_didnt_come.cnt_runners <= 404)]
y = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 139) & (come_with_didnt_come.cnt_runners <= 404)]

X = X.drop(columns=['is_repeated', 'athlete_id', 'record_id', 'event_name', 'event_date', 'region', 'city', 'is_newbie', 
    'athlete_gender', 'weather.main','weather.description','track_description_surface',
   'dew_point','wind_deg', 'clouds', 'temp',
    'is_male',
    'cnt_volunteers', 'rain.1h', 'snow.1h',

    'finish_time', 'pressure', 'humidity','wind_speed','feels_like', 'cnt_runners', 'athlete_age',

    #'humidity','finish_time', 'athlete_age', 'athlete_gender', 'wind_speed', 'feels_like', 
    #'pressure', 'temp', 
    # 'snow.1h', 'rain.1h', 'weather.main', 'weather.description', 'track_description_surface', 'wind_deg', 'clouds', 
    #'cnt_runners'
], axis=1)  
y = y['is_repeated']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42) 
#categorical_cols = ['athlete_gender', 'weather.main', 'weather.description', 'track_description_surface']

model = CatBoostClassifier(iterations=1000, depth=4, learning_rate=0.1,verbose=0 )# ,cat_features=categorical_cols)  
print('катбуст для группы 139-404 УЧАСТНИКОВ для волонтеров')
model.fit(X, y)  


preds = model.predict(X_test)  
cm = confusion_matrix(y_test, preds)
print(cm)
print(f'Accuracy: {accuracy_score(y_test, preds):.2f}') 




feature_importances = model.get_feature_importance()  
importance_df = pd.DataFrame({  
    'Feature': X.columns,  # Используйте реальные имена признаков  
    'Importance': feature_importances  
})  

# Сортировка значений важности в порядке убывания  
importance_df = importance_df.sort_values(by='Importance', ascending=False)  

display(importance_df)


катбуст для группы 139-404 УЧАСТНИКОВ для волонтеров
[[61 28]
 [48 50]]
Accuracy: 0.59


,Feature,Importance
20,Связи с общественностью,10.182321
26,Фотограф,9.475169
12,Пейсер,7.367730
27,Хранение и доставка оборудования,6.005918
19,Разное,5.976895
13,Подготовка мероприятия,5.728700
18,Раздача карточек позиций,5.462519
22,Сканирование штрих-кодов,5.411131
17,Проверка трассы,5.360033
15,Проведение разминки,4.157824


Берем из этого катбуста топ-10 волонтеров и контрольные переменные. модель для 3 группы 139-404 участников

In [15]:
X = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 139) & (come_with_didnt_come.cnt_runners <= 404)]
y = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 139) & (come_with_didnt_come.cnt_runners <= 404)]



need_columns = ['finish_time', 'athlete_age', 'feels_like', 'pressure', 'humidity', 'wind_speed', 
                'Связи с общественностью', 'Фотограф','Пейсер', 'Хранение и доставка оборудования','Разное', 'Подготовка мероприятия','Раздача карточек позиций',
                  'Сканирование штрих-кодов','Проверка трассы', 'Проведение разминки',
               # '', '','', '','', '','', '',
                  'cnt_runners'
                  ]
                






for col1 in need_columns:
    for col2 in need_columns:
        if col1 != col2:
            try:
                coef = X[col1].corr(X[col2],method='spearman')
                if abs(coef) > 0.8:
                    print('Возможна мультиколлинеарность: ', coef, col1, col2)
            except:
                print('ошибка в цикле на проверке корреляций между ', col1, " ", col2)

for_model = X[need_columns]


for_model = sm.add_constant(for_model)
y = y['is_repeated']
model = sm.Logit(y, for_model).fit()
print('\n')
print('Модель для предсказывания is_repeated новичка, в пробежках где было 139-404 людей')
print('\n')
print(model.summary())
marginal_effects = model.get_margeff()
print('\n')
print('Предельные эффекты:')
print(marginal_effects.summary())
print('\n')

y_pred = model.predict(for_model)
y_pred_class = (y_pred > 0.5).astype(int)  
cm = confusion_matrix(y, y_pred_class)
print(cm)
accuracy = (y_pred_class == y).mean()  
print(f'Accuracy: {accuracy:.2f}')  

Optimization terminated successfully.
         Current function value: 0.655668
         Iterations 5


Модель для предсказывания is_repeated новичка, в пробежках где было 139-404 людей


                           Logit Regression Results                           
Dep. Variable:            is_repeated   No. Observations:                 1865
Model:                          Logit   Df Residuals:                     1847
Method:                           MLE   Df Model:                           17
Date:                Tue, 10 Dec 2024   Pseudo R-squ.:                 0.05383
Time:                        14:03:17   Log-Likelihood:                -1222.8
converged:                       True   LL-Null:                       -1292.4
Covariance Type:            nonrobust   LLR p-value:                 3.205e-21
                                       coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------